# Machine Learning: Text Classification Assignment

In [42]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split as tts
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [21]:
PATH = '/content/drive/MyDrive/AP_News/'

DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'([\w_\s]+)/.*'

corpus = CategorizedPlaintextCorpusReader(PATH, DOC_PATTERN, cat_pattern=CAT_PATTERN)

In [23]:
corpus.fileids()[0]

'health/http-apnews-com-03bc406312384416843138b2b23dec14.txt'

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [24]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]

In [26]:
categories = [corpus.categories(fileid)[0] 
              for fileid in corpus.fileids()]

In [27]:
categories[0]

'health'

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [46]:
def preprocess(docs):
  lemmatizer = WordNetLemmatizer()
  stemmer = SnowballStemmer('english')
  preprocessed = []

  for doc in docs:
    tokenized = word_tokenize(doc)

    cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower())) for token in tokenized
               if not token.lower() in stopwords.words('english')
               if token.isalpha()
               ]
    untokenized = " ".join(cleaned)
    preprocessed.append(untokenized)
  
  return preprocessed

In [47]:
preprocessed = preprocess(docs)

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [41]:
X_train, X_test, y_train, y_test = tts(preprocessed, categories, test_size=0.3)

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [43]:
model = Pipeline([
                  ('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression()),
])

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [44]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

      health       0.89      0.84      0.86        19
    politics       0.93      0.78      0.85        18
      sports       0.94      1.00      0.97        17
        tech       0.67      0.83      0.74        12

    accuracy                           0.86        66
   macro avg       0.86      0.86      0.86        66
weighted avg       0.87      0.86      0.87        66



### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [45]:
scores = cross_val_score(model, preprocessed, categories, cv=10, scoring='f1_macro')
scores.mean()

0.8187741262005968

### Ingest, preprocess, and predict the topic of the article at the following URL.

In [48]:
url = 'https://www.nytimes.com/2019/11/25/business/uber-london.html'

In [59]:
import requests
from bs4 import BeautifulSoup

def get_url_text(url):
  response = requests.get(url)
  content = response.text

  TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']
  soup = BeautifulSoup(content, "lxml")
  text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
  text = ' '.join(text_list)
  return text.strip().replace('\n', '')

In [61]:
text = get_url_text(url)
cleaned = preprocess([text])
model.predict(cleaned)[0]

'tech'